In [2]:
from constants.abs_paths import AbsDirPath
from processing_pipeline.model.CSVDFHandler import CSVDFHandler
from processing_pipeline.utilities.data_transformation import load_all_files
from utilities.csv.split_file_into_n_parts import split_file

In [3]:
from processing_pipeline.data_optimization.stage_cfg import S0NoiseFiltering

current_stage_cfg = S0NoiseFiltering
current_stage_cfg

StageConfig(name='s0', columns_in=['sentence'], in_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s0_noise_filtering'), out_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s0_noise_filtering_o'), Stage=<processing_pipeline.s0_noise_filtering.NoiseFiltering_v2.NoiseFilteringStage_v2 object at 0x0000022A465BF7D0>, next_stage=StageConfig(name='s1', columns_in=['qa', 'sentence'], in_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s1_qa_relevance_check'), out_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s1_qa_relevance_check_o'), Stage=<processing_pipeline.s1_qa_relevance_check.QARelevanceCheck_v2.QARelevanceCheckStage_v2 object at 0x0000022A465BE060>, next_stage=StageConfig(name='s2', columns_in=['sentence'], in_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/k

In [6]:
current_stage_cfg = current_stage_cfg.next_stage
current_stage_cfg

StageConfig(name='s3', columns_in=['qa', 'sentence'], in_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s3_tactic_extraction'), out_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/merged'), Stage=<processing_pipeline.s3_tactic_extraction.TacticExtraction_v2.TacticExtractionStage_v2 object at 0x0000022A465BC200>, next_stage=None, _apply_filters=<function <lambda> at 0x0000022A46558D60>, _on_load=<function StageConfig.<lambda>.<locals>.<lambda> at 0x0000022A4659BF60>)

In [7]:
in_dir = current_stage_cfg.in_dir
out_path = AbsDirPath.SAMPLES / (current_stage_cfg.name + ".csv")
in_dir, out_path

(WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s3_tactic_extraction'),
 WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/samples/s3.csv'))

In [7]:
df = load_all_files(in_dir)
df.shape

Loaded 44 files, [WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s3_tactic_extraction/allenai.scispacy.v0.5.5.code_comment.parquet'), WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s3_tactic_extraction/allenai.scispacy.v0.5.5.docs.parquet'), WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s3_tactic_extraction/allenai.scispacy.v0.5.5.issue.parquet'), WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s3_tactic_extraction/allenai.scispacy.v0.5.5.issue_comment.parquet'), WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s3_tactic_extraction/allenai.scispacy.v0.5.5.release.parquet'), WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords_2/s3_tactic_extraction/google.deepvariant.v1.6.1.code_comment.parquet'), WindowsPath('C:/Users/Dmitrijs/Do

(28230, 10)

In [8]:
current_stage_cfg.columns_out

['s3_architectural_activity_extraction',
 's3_core_concept_analysis',
 's3_is_tactic_relevant',
 's3_relevance_reason',
 's3_tactic_evaluation',
 's3_selected_tactic',
 's3_justification']

In [9]:
for col in current_stage_cfg.columns_out:
    try:
        df = df[df[col].str.len() > 0]
    except:
        pass
df.shape

(26954, 10)

In [10]:
sample_size = 384

In [11]:
# # fake
# sample_size = df.shape[0]

In [13]:
out_path.parent.mkdir(exist_ok=True)
df.sample(sample_size).to_csv(out_path)


## Optimize for parallel processing?

In [12]:
n_parts = 5
split_file(out_path, n_parts, CSVDFHandler())

## Add tactic descriptions for s3

In [8]:
import pandas as pd
from cfg.tactics.tactic_description import tactic_descriptions

In [28]:
path_to_verified = AbsDirPath.SAMPLES_VERIFIED / out_path.name

In [29]:
df = pd.read_csv(path_to_verified)
df.shape

(384, 14)

In [30]:
df["s3_tactic_desc"] = df.s3_selected_tactic.apply(lambda x: tactic_descriptions.get(x, pd.NA))
df.head()

,Unnamed: 0,qa,sentence,s3_prompt,s3_architectural_activity_extraction,s3_core_concept_analysis,s3_is_tactic_relevant,s3_relevance_reason,s3_tactic_evaluation,s3_selected_tactic,s3_justification,s3_v_prompt,s3_v_evaluation,s3_v_reasoning,s3_tactic_desc
0,27922,testability,"rder, our approach is pinning it temporarily u...",\r\nBased on the rules provided in the system ...,"rder, our approach is pinning it temporarily u...",The primary architectural concept identified i...,True,The concept identified is central to understan...,true,NaN,The concept identified in this text is version...,"Now, perform your your audit based on the data...",correct,The executor's claim that the discussion about...,<NA>
1,4089,modifiability,Hi @moldach . Thank you for reporting this! Fo...,\r\nBased on the rules provided in the system ...,I drafted a few new sections in the Quick Star...,Split Module,True,The extracted sentence discusses a specific ac...,true,Split Module,The extracted sentence clearly indicates that ...,"Now, perform your your audit based on the data...",correct,The executor's claim in Step 1 is that they ar...,Refining a module into several smaller modules...
2,7174,deployability,API extraction should not generate an actual p...,\r\nBased on the rules provided in the system ...,API extraction should not generate an actual p...,The primary architectural concept is preventin...,True,The extracted activity prevents generating a p...,split module,Split Module,The selected tactic 'split module' best matche...,"Now, perform your your audit based on the data...",correct,The executor's claim that the discussion on de...,Refining a module into several smaller modules...
3,5247,usability,"Hi Masaru,. I've filed an internal issue to tr...",\r\nBased on the rules provided in the system ...,"Hi Masaru,. I've filed an internal issue to tr...",We'll keep usability for beginners in mind for...,True,The text discusses a design decision related t...,Maintain system models,NaN,The selected tactic 'Maintain system models' d...,"Now, perform your your audit based on the data...",correct,The executor's claim that the text describes a...,<NA>
4,2321,availability,rent from the make_examples command you posted...,\r\nBased on the rules provided in the system ...,I tested with `sudo docker run -it gcr.io/deep...,The primary architectural concept identified i...,True,The extracted sentence discusses a concrete te...,true,Ping/Echo,The extracted sentence discusses the use of Do...,"Now, perform your your audit based on the data...",correct,The executor's claim that the discussion on Do...,An asynchronous request/response message pair ...


In [31]:
df.to_csv(path_to_verified, index=False)